In [48]:
import pandas as pd
import numpy as np

In [49]:
df = pd.read_csv("../Cleaning_Engineering/added_demand_estimates_by_destination.csv")

In [50]:
df['dynamic_price'] = df['base_price'] * (1 + df['estimated_eta'] * df['estimated_demand'])

In [51]:
df

,cab_type,source,destination,car_type,date_time,weekday,rush_hour,is_raining,temp_groups,surge_multiplier,price,base_price,estimated_eta,estimated_a,estimated_b,estimated_demand,dynamic_price
0,Uber,South Station,Theatre District,Wheel Chair Accessible,2018-11-26 04:43:18.387,1,0,0,40,1.0,9.5,6.5,0.224242,0.057036,8.622001,8.656428,19.117349
1,Uber,South Station,Theatre District,Wheel Chair Accessible,2018-11-26 04:49:39.601,1,0,0,40,1.0,9.5,6.5,0.224242,0.057036,8.622001,8.656428,19.117349
2,Uber,South Station,Theatre District,Wheel Chair Accessible,2018-11-26 06:25:02.313,1,0,0,40,1.0,8.5,6.5,0.224242,0.057036,8.622001,8.657298,19.118616
3,Uber,South Station,Theatre District,Wheel Chair Accessible,2018-11-26 06:30:02.379,1,0,0,40,1.0,7.0,6.5,0.224242,0.057036,8.622001,8.658868,19.120905
4,Uber,South Station,Theatre District,Wheel Chair Accessible,2018-11-26 07:16:52.978,1,1,0,40,1.0,7.0,6.5,0.224242,0.057036,8.622001,8.658868,19.120905
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574173,Uber,Northeastern University,Back Bay,Luxury SUV,2018-12-17 04:30:06.790,1,0,0,30,1.0,25.0,25.0,0.000002,0.999873,19.999876,20.999741,25.001256
574174,Lyft,Northeastern University,Back Bay,Luxury SUV,2018-12-17 05:00:04.215,1,0,0,30,1.0,26.0,25.0,0.000002,0.999873,19.999876,20.999741,25.001256
574175,Uber,Northeastern University,Back Bay,Luxury SUV,2018-12-17 05:20:10.955,1,0,0,30,1.0,26.5,25.0,0.000002,0.999873,19.999876,20.999741,25.001256
574176,Uber,Northeastern University,Back Bay,Luxury SUV,2018-12-17 05:50:10.584,1,0,0,30,1.0,26.5,25.0,0.000002,0.999873,19.999876,20.999741,25.001256


Equations

Q(Quantity Demanded)= a ⋅P^−η + b

Price = Base Price ×(1 +η × Demand Factor)


Limitations: It is said that the problem we are facing is due to the low value of a. However, if we assume a large value from the beginning and run MCMC, it hardly works due to memory issues. So, we estimate a to a lower decimal point, calculate the A and demand, and then multiply it by a certain amount of constant (according to market conditions).



Solution: We ended up multiplying the current DEMAND column by a constant, which we’re told is a common practice for companies to keep A around 100. (We’re told that this number can be set arbitrarily by the company.) However, our current estimate is that A is filled with a number like 0.24242. So why not multiply A by a constant (around 600), make A around 100, and then recalculate DEMAND to apply your modeling?

In [52]:
df['estimated_a'].describe()


count    574178.000000
mean          0.320383
std           0.438718
min           0.005266
25%           0.017621
50%           0.038291
75%           0.999721
max           0.999968
Name: estimated_a, dtype: float64

create visual for intial data showing the lack of diversity in demand values, trying constant 600

In [53]:
adj_df = df.copy()
const = 600
adj_df['estimated_a'] = adj_df['estimated_a']* const

In [54]:
# Assuming base_price, elasticity, and demand are columns in a DataFrame named df
adj_df['estimated_demand'] = adj_df['estimated_a'] * np.exp(-np.abs(adj_df['estimated_eta']) * np.log(adj_df['price'])) + adj_df['estimated_b']
adj_df['dynamic_price'] = adj_df['base_price'] * (1 + adj_df['estimated_eta'] * adj_df['estimated_demand'])



In [55]:
df[(df['source'] == 'South Station') & (df['destination'] == 'Theatre District') & (df['car_type'] == 'Base')]['estimated_demand'].describe()


count    1337.000000
mean        8.221610
std         0.000629
min         8.219048
25%         8.221185
50%         8.221657
75%         8.222206
max         8.222518
Name: estimated_demand, dtype: float64

In [56]:
adj_df[(adj_df['source'] == 'South Station') & (adj_df['destination'] == 'Theatre District') & (adj_df['car_type'] == 'Base')]['estimated_demand'].describe()

count    1337.000000
mean       19.276486
std         0.377641
min        17.739701
25%        19.021680
50%        19.304650
75%        19.634417
max        19.821641
Name: estimated_demand, dtype: float64

In [59]:
df[(df['source'] == 'South Station') & (df['destination'] == 'Theatre District') & (df['car_type'] == 'Base')]['price'].describe()


count    1337.000000
mean        8.223635
std         1.427419
min         6.500000
25%         7.000000
50%         8.000000
75%         9.000000
max        16.000000
Name: price, dtype: float64

In [60]:
df[(df['source'] == 'South Station') & (df['destination'] == 'Theatre District') & (df['car_type'] == 'Base')]['dynamic_price'].describe()


count    1337.000000
mean       18.214960
std         0.000897
min        18.211311
25%        18.214355
50%        18.215027
75%        18.215810
max        18.216255
Name: dynamic_price, dtype: float64

In [61]:
adj_df[(adj_df['source'] == 'South Station') & (adj_df['destination'] == 'Theatre District') & (adj_df['car_type'] == 'Base')]['dynamic_price'].describe()

count    1337.000000
mean       33.967038
std         0.538101
min        31.777275
25%        33.603965
50%        34.007168
75%        34.477053
max        34.743829
Name: dynamic_price, dtype: float64

In [57]:
adj_df

,cab_type,source,destination,car_type,date_time,weekday,rush_hour,is_raining,temp_groups,surge_multiplier,price,base_price,estimated_eta,estimated_a,estimated_b,estimated_demand,dynamic_price
0,Uber,South Station,Theatre District,Wheel Chair Accessible,2018-11-26 04:43:18.387,1,0,0,40,1.0,9.5,6.5,0.224242,34.221533,8.622001,29.278334,49.175217
1,Uber,South Station,Theatre District,Wheel Chair Accessible,2018-11-26 04:49:39.601,1,0,0,40,1.0,9.5,6.5,0.224242,34.221533,8.622001,29.278334,49.175217
2,Uber,South Station,Theatre District,Wheel Chair Accessible,2018-11-26 06:25:02.313,1,0,0,40,1.0,8.5,6.5,0.224242,34.221533,8.622001,29.800011,49.935597
3,Uber,South Station,Theatre District,Wheel Chair Accessible,2018-11-26 06:30:02.379,1,0,0,40,1.0,7.0,6.5,0.224242,34.221533,8.622001,30.742422,51.309228
4,Uber,South Station,Theatre District,Wheel Chair Accessible,2018-11-26 07:16:52.978,1,1,0,40,1.0,7.0,6.5,0.224242,34.221533,8.622001,30.742422,51.309228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574173,Uber,Northeastern University,Back Bay,Luxury SUV,2018-12-17 04:30:06.790,1,0,0,30,1.0,25.0,25.0,0.000002,599.924015,19.999876,619.919271,25.037071
574174,Lyft,Northeastern University,Back Bay,Luxury SUV,2018-12-17 05:00:04.215,1,0,0,30,1.0,26.0,25.0,0.000002,599.924015,19.999876,619.919215,25.037071
574175,Uber,Northeastern University,Back Bay,Luxury SUV,2018-12-17 05:20:10.955,1,0,0,30,1.0,26.5,25.0,0.000002,599.924015,19.999876,619.919187,25.037071
574176,Uber,Northeastern University,Back Bay,Luxury SUV,2018-12-17 05:50:10.584,1,0,0,30,1.0,26.5,25.0,0.000002,599.924015,19.999876,619.919187,25.037071


In [58]:
df

,cab_type,source,destination,car_type,date_time,weekday,rush_hour,is_raining,temp_groups,surge_multiplier,price,base_price,estimated_eta,estimated_a,estimated_b,estimated_demand,dynamic_price
0,Uber,South Station,Theatre District,Wheel Chair Accessible,2018-11-26 04:43:18.387,1,0,0,40,1.0,9.5,6.5,0.224242,0.057036,8.622001,8.656428,19.117349
1,Uber,South Station,Theatre District,Wheel Chair Accessible,2018-11-26 04:49:39.601,1,0,0,40,1.0,9.5,6.5,0.224242,0.057036,8.622001,8.656428,19.117349
2,Uber,South Station,Theatre District,Wheel Chair Accessible,2018-11-26 06:25:02.313,1,0,0,40,1.0,8.5,6.5,0.224242,0.057036,8.622001,8.657298,19.118616
3,Uber,South Station,Theatre District,Wheel Chair Accessible,2018-11-26 06:30:02.379,1,0,0,40,1.0,7.0,6.5,0.224242,0.057036,8.622001,8.658868,19.120905
4,Uber,South Station,Theatre District,Wheel Chair Accessible,2018-11-26 07:16:52.978,1,1,0,40,1.0,7.0,6.5,0.224242,0.057036,8.622001,8.658868,19.120905
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574173,Uber,Northeastern University,Back Bay,Luxury SUV,2018-12-17 04:30:06.790,1,0,0,30,1.0,25.0,25.0,0.000002,0.999873,19.999876,20.999741,25.001256
574174,Lyft,Northeastern University,Back Bay,Luxury SUV,2018-12-17 05:00:04.215,1,0,0,30,1.0,26.0,25.0,0.000002,0.999873,19.999876,20.999741,25.001256
574175,Uber,Northeastern University,Back Bay,Luxury SUV,2018-12-17 05:20:10.955,1,0,0,30,1.0,26.5,25.0,0.000002,0.999873,19.999876,20.999741,25.001256
574176,Uber,Northeastern University,Back Bay,Luxury SUV,2018-12-17 05:50:10.584,1,0,0,30,1.0,26.5,25.0,0.000002,0.999873,19.999876,20.999741,25.001256
